In [ ]:
# import packages
import collections
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
import math
from moviepy.editor import VideoFileClip
%matplotlib inline



# Choose the number of sliding windows
NUM_WINDOWS = 10
# Set the width of the windows +/- margin
MARGIN = 50
# Set minimum number of pixels found to recenter window
MIN_NUM_PIXELS = 50

# Perspective transform matrix
MTX_PT = None
MTX_PT_INV = None
# Undistortion
MTX_UD = None
DIST_UD = None


def set_globals(M, M_inv, mtx, dist):
    global MTX_PT
    global MTX_PT_INV
    global MTX_UD
    global DIST_UD
    
    MTX_PT = M
    MTX_PT_INV = M_inv
    MTX_UD = mtx
    DIST_UD = dist

    
def find_lane_pixels_raw(binary_warped):
    """
    Find lane pixels using histogram and sliding window algorithm
    """
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    one_third = np.int(histogram.shape[0] // 3)
    two_third = np.int(histogram.shape[0] * 2 // 3)
    leftx_base = np.argmax(histogram[:one_third])
    rightx_base = np.argmax(histogram[two_third:]) + two_third

    # Set height of windows - based on NUM_WINDOWS above and image shape
    window_height = np.int(binary_warped.shape[0] // NUM_WINDOWS)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in NUM_WINDOWS
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(NUM_WINDOWS):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - MARGIN
        win_xleft_high = leftx_current + MARGIN
        win_xright_low = rightx_current - MARGIN
        win_xright_high = rightx_current + MARGIN
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) 
                          & (nonzeroy < win_y_high) 
                          & (nonzerox >= win_xleft_low)
                          & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) 
                           & (nonzeroy < win_y_high) 
                           & (nonzerox >= win_xright_low) 
                           & (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > MIN_NUM_PIXELS pixels, recenter next window 
        # on their mean position
        if len(good_left_inds) > MIN_NUM_PIXELS:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > MIN_NUM_PIXELS:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty


def find_lane_pixels_opt(binary_warped, left_fit, right_fit):
    """
    Find lane pixels based on previous polynomial fit
    """
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    left_x_prediction = left_fit[0]*nonzeroy**2 + left_fit[1]*nonzeroy + left_fit[2]
    left_coordinates = ((nonzerox >= left_x_prediction - MARGIN) 
                        & (nonzerox <= left_x_prediction + MARGIN)).nonzero()[0]

    right_x_prediction = right_fit[0]*nonzeroy**2 + right_fit[1]*nonzeroy + right_fit[2]
    right_coordinates = ((nonzerox >= right_x_prediction - MARGIN) 
                         & (nonzerox <= right_x_prediction + MARGIN)).nonzero()[0]
    
    leftx = nonzerox[left_coordinates]
    lefty = nonzeroy[left_coordinates]
    rightx = nonzerox[right_coordinates]
    righty = nonzeroy[right_coordinates]
    
    return leftx, lefty, rightx, righty
    

def fit_polynomial(leftx, lefty, rightx, righty):
    """
    Fit polynomial based on the given points
    """
    if leftx.size == 0:
        left_fit = None
    else:
        left_fit = np.polyfit(lefty, leftx, 2)
    if rightx.size == 0:
        right_fit = None
    else:
        right_fit = np.polyfit(righty, rightx, 2)
    return left_fit, right_fit


def generate_lines(left_fit, right_fit, img_height):
    """
    Generate fitted lines based
    """
    left_fitx = None
    right_fitx = None
    ploty = np.linspace(0, img_height - 1, img_height)
    if left_fit != None:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    if right_fit != None:
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fitx, right_fitx, ploty


def plot_detection(binary_warped, left_fitx, right_fitx, ploty):
    """
    Plot the detection areas enclosed by the two lanes
    """
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    if left_fitx == None or right_fitx == None:
        return out_img
    
    left_line_window = np.array(np.transpose(np.vstack([left_fitx, ploty])))
    right_line_window = np.array(np.flipud(np.transpose(np.vstack([right_fitx, ploty]))))
    line_points = np.vstack((left_line_window, right_line_window))
    cv2.fillPoly(out_img, np.int_([line_points]), [0,255, 0])
    unwarped = cv2.warpPerspective(out_img, MTX_PT_INV, 
                                   get_shape_of_image(out_img), 
                                   flags=cv2.INTER_LINEAR)
    return unwarped
    

def get_average_lines(past_leftx, past_right_x, left_fitx, right_fitx, size=10):
    """
    Get average lines based on history
    """
    if left_fitx != None:
        past_leftx.append(left_fitx)
        if len(past_leftx) == size:
            past_leftx.popleft()
    
    if right_fitx != None:
        past_rightx.append(right_fitx)
        if len(past_rightx) == size:
            past_rightx.popleft()
    
    left_next = None
    right_next = None
    
    if past_leftx:
        left_next = np.zeros_like(past_leftx[-1])
        total = 0
        for i, line in enumerate(past_leftx):
            left_next += (i + 1) * line
            total += i + 1
        left_next /= total
        
    if past_rightx:
        right_next = np.zeros_like(past_rightx[-1])
        total = 0
        for i, line in enumerate(past_rightx):
            right_next += (i + 1) * line
            total += i + 1
        right_next /= total

    return left_next, right_next


def measure_radius_of_curvature(x_values, height):
    """
    Calculate the radius of curvature in meters
    """
    if x_values.size == 0:
        return -float('inf')
    # meters per pixel in y dimension
    ym_per_pix = 30/720 
    # meters per pixel in x dimension
    xm_per_pix = 3.7/700 
    # If no pixels were found return None
    y_points = np.linspace(0, height-1, height)
    y_eval = np.max(y_points)
    # Fit new polynomials to x,y in world space
    fit_cr = np.polyfit(y_points*ym_per_pix, x_values*xm_per_pix, 2)
    curverad = ((1 + (2*fit_cr[0]*y_eval*ym_per_pix + fit_cr[1])**2)**1.5) / np.absolute(2*fit_cr[0])
    
    return curverad

In [1]:
def find_lane_lines_in_image(img):
    """
    Fit polynomial for lane pixels on a warped binary image
    """
    thresholded_img = mix_threshold(undistort_one(img, MTX_UD, DIST_UD), True)
    binary_warped = warp(thresholded_img, M)
    
    # Find our lane pixels first
    leftx, lefty, rightx, righty = find_lane_pixels_raw(binary_warped)
    
    left_fit, right_fit = fit_polynomial(leftx, lefty, 
                                         rightx, righty)
    # fit polynomial
    left_fitx, right_fitx, ploty = generate_lines(left_fit, right_fit, 
                                                  binary_warped.shape[0])
    
    left_curve_rad = measure_radius_of_curvature(left_fitx, binary_warped.shape[0])
    right_curve_rad = measure_radius_of_curvature(right_fitx, binary_warped.shape[0])
    average_curve_rad = (left_curve_rad + right_curve_rad) / 2.0
    
    if average_curve_rad < 0:
        curvature_string = "Radius of curvature: NA"
        offset_string = "Center offset: NA"
    else:
        curvature_string = "Radius of curvature: %.0f m" % average_curve_rad
        # compute the offset from the center
        lane_center = (right_fitx[-1] + left_fitx[-1]) / 2.0
        # meters per pixel in x dimension
        xm_per_pix = 3.7/700 
        center_offset_pixels = binary_warped.shape[1] / 2.0 - lane_center
        center_offset_mtrs = xm_per_pix * center_offset_pixels
        offset_string = "Center offset: %.2f m" % center_offset_mtrs

    detection = plot_detection(binary_warped, left_fitx, right_fitx, ploty)
    combined_image = cv2.addWeighted(img, 1, detection, 0.3, 0)
    cv2.putText(combined_image,curvature_string, (100, 90), 
                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), thickness=4)
    cv2.putText(combined_image, offset_string, (100, 150), 
                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), thickness=4)
    return combined_image
    

In [ ]:
CURVATURE_MARGIN = 500

past_leftx = collections.deque()
past_rightx=collections.deque()
past_curve_rad = -1.0
left_fit=None 
right_fit=None
bad_frame = True

def find_lane_lines_in_video(img):
    """
    Fit polynomial for lane pixels on a warped binary image
    """
    global past_leftx
    global past_rightx
    global past_curve_rad
    global left_fit
    global right_fit
    global bad_frame
    
    thresholded_img = mix_threshold(undistort_one(img, MTX_UD, DIST_UD), True)
    binary_warped = warp(thresholded_img, MTX_PT)
    
    # Find our lane pixels first
    # for simplicity, if left_fit is None, right_fit is also None here
    if left_fit == None or right_fit == None or bad_frame:
        leftx, lefty, rightx, righty = find_lane_pixels_raw(binary_warped)

        left_fit, right_fit = fit_polynomial(leftx, lefty, 
                                             rightx, righty)
    else:
        leftx, lefty, rightx, righty = find_lane_pixels_opt(binary_warped, 
                                                            left_fit, right_fit)
        left_fit, right_fit = fit_polynomial(leftx, lefty, 
                                             rightx, righty)

    # fit polynomial
    left_fitx, right_fitx, ploty = generate_lines(left_fit, right_fit, 
                                                  binary_warped.shape[0])

    bad_frame = False
    
    left_curve_rad = measure_radius_of_curvature(left_fitx, binary_warped.shape[0])
    right_curve_rad = measure_radius_of_curvature(right_fitx, binary_warped.shape[0])
    if abs(left_curve_rad - right_curve_rad) > CURVATURE_MARGIN:
        bad_frame = True
        if past_curve_rad > 0:
            if abs(left_curve_rad - past_curve_rad) < abs(right_curve_rad - past_curve_rad):
                average_curve_rad = (left_curve_rad + past_curve_rad) / 2.0
            elif abs(left_curve_rad - past_curve_rad) > abs(right_curve_rad - past_curve_rad):
                average_curve_rad = (right_curve_rad + past_curve_rad) / 2.0
        else:
            if leftx.size > 3* rightx.size:
                average_curve_rad = left_curve_rad
            elif 3 * leftx.size < rightx.size:
                average_curve_rad = right_curve_rad
            else:
                average_curve_rad = (right_curve_rad + left_curve_rad) / 2.0
    elif past_curve_rad > 0:
        average_curve_rad = (left_curve_rad + right_curve_rad + past_curve_rad) / 3.0
    else:
        average_curve_rad = (left_curve_rad + right_curve_rad) / 2.0
        
    past_curve_rad = average_curve_rad

    if average_curve_rad < 0:
        curvature_string = "Radius of curvature: NA"
        offset_string = "Center offset: NA"        
    else:
        curvature_string = "Radius of curvature: %.0f m" % average_curve_rad
        if right_fitx != None and left_fitx != None:
            bottom_width = right_fitx[-1] - left_fitx[-1]
            if bottom_width > 750 or bottom_width < 450:
                bad_frame = True
            # compute the offset from the center
            lane_center = (right_fitx[-1] + left_fitx[-1]) / 2.0
            # meters per pixel in x dimension
            xm_per_pix = 3.7/700 
            center_offset_pixels = binary_warped.shape[1] / 2.0 - lane_center
            center_offset_mtrs = xm_per_pix * center_offset_pixels
            offset_string = "Center offset: %.2f m" % center_offset_mtrs
        else:
            offset_string = "Center offset: NA"
            
    
    left_avg, right_avg = get_average_lines(past_leftx, past_rightx, 
                                            left_fitx, right_fitx)
    
    detection = plot_detection(binary_warped, left_avg, right_avg, ploty)
    combined_image = cv2.addWeighted(img, 1, detection, 0.3, 0)
    cv2.putText(combined_image,curvature_string, (100, 90), 
                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), thickness=4)
    cv2.putText(combined_image, offset_string, (100, 150), 
                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), thickness=4)
    return combined_image